# Generating Vegan Recipes

!!! note
    To download this example as a Jupyter notebook, click [here](https://github.com/ShreyaR/guardrails/blob/main/docs/examples/recipe_generation.ipynb).

In this example, we will use Guardrails to generate vegan mac and cheese recipe.

## Objective

We want to generate a vegan Mac-n-Cheese recipe as a list of ingredients and instructions. We will use Guardrails to make sure the recipe is vegan.

In [ ]:
import guardrails as gd
from rich import print

## Step 1: Create the RAIL Spec

Ordinarily, we would create an RAIL spec in a separate file. For the purposes of this example, we will create the spec in this notebook as a string following the RAIL syntax. For more information on RAIL, see the [RAIL documentation](../rail/output.md).

Here, we request:



In [ ]:
rail_str = """
<rail version="0.1">

<script language='python'>
from dataclasses import dataclass
from guardrails.validators import Validator, EventDetail, register_validator

import re
from typing import Dict, List

NON_VEGAN_INGREDIENTS = ["butter", "milk", "eggs", "cheese", "cream", "yogurt"]
SUBSTITUTIONS = {
    "butter": "margarine",
    "milk": "soy milk",
    "eggs": "flax eggs",
    "cheese": "vegan cheese",
    "cream": "soy cream",
    "yogurt": "soy yogurt",
}

@register_validator(name="is-vegan", data_type="string")
class IsVegan(Validator):

    def validate(self, key, value, schema) -> Dict:
        global NON_VEGAN_INGREDIENTS, SUBSTITUTIONS

        # Make sure the ingredient is not in the list of non-vegan ingredients.
        if value.lower() in NON_VEGAN_INGREDIENTS:
            raise EventDetail(
                key,
                value,
                schema,
                f"Value ${value} is not vegan.",
                # Programmatically fix the value by replacing it with a vegan
                # substitute.
                SUBSTITUTIONS[value.lower()],
            )

        return schema
</script>


<output>
    <list name="ingredients" description="What are the ingredients for the recipe?">
        <object>
            <integer name="index" format="1-indexed" />
            <string name="name" format="is-vegan" on-fail-is-vegan="fix" />
            <string name="brand" description="Suggested brand for the ingredient (if any)" />
            <bool name="optional" description="Is the ingredient necessary?" />
            <float name="quantity" format="units-imperial" />
            <string name="units" format="units-imperial" />
        </object>
    </list>
    <list name="instructions" description="What are the instructions for the recipe?">
        <object>
            <integer name="index" format="1-indexed" />
            <string name="step" />
        </object>
    </list>
</output>


<prompt>
Generate a recipe for vegan mac and cheese.
${gr.complete_json_suffix}
</prompt>

</rail>
"""

!!! note
    Here, we create a custom `IsVegan` validator that checks if the ingredient is vegan.
    We also set `on-fail-is-vegan` to `fix`, which in this case means that programatically we will replace the ingredient with a vegan substitute.

## Step 2: Create a `Guard` object with the RAIL Spec

We create a `gd.Guard` object that will check, validate and correct the output of the LLM. This object:

1. Enforces the quality criteria specified in the RAIL spec.
2. Takes corrective action when the quality criteria are not met.
3. Compiles the schema and type info from the RAIL spec and adds it to the prompt.

In [ ]:
guard = gd.Guard.from_rail_string(rail_str)

As we can see, a few formatters weren't supported. These formatters won't be enforced in the output, but this information can still be used to generate a prompt.

We see the prompt that will be sent to the LLM. The `{document}` is substituted with the user provided value at runtime.

In [ ]:
print(guard.base_prompt)

## Step 3: Wrap the LLM API call with `Guard`

In [ ]:
import openai

raw_llm_response, validated_response = guard(
    openai.Completion.create, engine="text-davinci-003", max_tokens=2048, temperature=0
)

The `guard` wrapper returns the raw_llm_respose (which is a simple string), and the validated and corrected output (which is a dictionary).

We can see that the output is a dictionary with the correct schema and types.

In [ ]:
print(validated_response)

In [ ]:
print(guard.state.most_recent_call.tree)